<a href="https://colab.research.google.com/github/joaramirezra/ClasificadorHojasCanabis/blob/master/PresentacionFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deteccion de anomalias en hojas de canabis 
### Febrero 2020
**Autores:**
- Johan Ramirez 
- Adriana Rivera

In [0]:
# Importacion de librerias
import cv2                                                
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from IPython.display import Image
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
